# Estimate Compression Ratio

This notebook estimates the compression ratio of the CompressionReduction module using data from other detectors which was manipulated to look more like the expected data from this detector.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
from math import ceil
import ipywidgets as widgets

from utils import *

Declare which files we want to estimate the compression ratio with. They should be the names of the `.bin` files in the `data/` folder. They can be obtained [here](https://anl.box.com/s/folizdpti1i95oz7yxclvheoysq5fmhf).

In [4]:
data_files = ["pilatus", "ptychography", "xpcs"]

## Make plots for one of the files

In [5]:
dropdown = widgets.Dropdown(
    options=[(n, i) for i, n in enumerate(data_files)],
    value=0,
    description='File:',
)
dropdown

Dropdown(description='File:', options=(('pilatus', 0), ('ptychography', 1), ('xpcs', 2)), value=0)

In [6]:
filename = data_files[dropdown.value]
print(f'Estimating compression using {filename} data')

Estimating compression using pilatus data


In [7]:
@jit(nopython=True)
def get_reduced_size(shift: np.ndarray, use_poisson: bool = True) -> int:
    compressed = np.zeros((64, 8 if use_poisson else 11), dtype=np.uint64)
    i = 0
    for row in range(0, 128, 4):
        for col in range(0, 8, 4):
            data = shift[col:col+4, row:row+4].flatten()
            poisson_encoded = poisson_encode(data)
            compressed_elem = length_shuffle_compress(poisson_encoded, bits_per_pixel = 7 if use_poisson else 10)
            compressed[i][0:compressed_elem.shape[0]] = compressed_elem
            i += 1
    [h_2bit, h_3bit, reduced] = reduce_data(compressed, maxblocks = 128, data_len = 7 if use_poisson else 10)

    return h_2bit.shape[0]*2 + ceil(h_3bit.shape[0]*(3 if use_poisson else 4)/16)*16 + reduced.shape[0]*16

def get_reduced_sizes(filename: str, use_poisson: bool = True):
    reduced_sizes = []
    for frame in get_frames(f'data/{filename}.bin'):
        for shift_col in range(0, 128, 8):
            shift = frame[shift_col:shift_col+8]
            reduced_sizes.append(get_reduced_size(shift, use_poisson=use_poisson))
    return reduced_sizes


In [8]:
# Get the size (in bits) of every reduced shift in every frame
reduced_sizes = get_reduced_sizes(filename)

uncompressed_size = 128*8*10

%matplotlib widget
plt.plot([uncompressed_size/s for s in reduced_sizes], label="Compression Ratio")
plt.legend()
plt.xlabel("Shift Number")
plt.ylabel("Compression Ratio")
plt.title("Compression Ratio after Reduction for every shift")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def get_fifo_write_sizes(reduced_sizes):
    fifo_write_sizes = []
    merge_buffer = 0
    for s in reduced_sizes:
        s = ceil(s/4)*4 # EnsureBlocks uses 64-bit elements instead of 16-bit

        # Check if we need to write to FIFO
        if merge_buffer + s > (1024 - 8)*10:
            fifo_write_sizes.append(ceil(merge_buffer/(1024-8))*1024)
            merge_buffer = 0
        else:
            fifo_write_sizes.append(0)

        merge_buffer += s
    return fifo_write_sizes

fifo_write_sizes = get_fifo_write_sizes(reduced_sizes)

%matplotlib widget
plt.plot(fifo_write_sizes, label="FIFO write sizes")
plt.legend()
plt.xlabel("Tick Number")
plt.ylabel("# bits")
plt.title("FIFO write size for every tick")
plt.show()            

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Overall compression ratios

In [10]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [11]:
for filename in data_files:
    printmd(f'### Estimated using {filename} data')

    reduced = get_reduced_sizes(filename)
    fifo_sizes = get_fifo_write_sizes(reduced)
    print(f"With poisson encoding: {uncompressed_size*len(fifo_sizes)/sum(fifo_sizes)}")

    reduced_np = get_reduced_sizes(filename, use_poisson=False)
    fifo_sizes_np = get_fifo_write_sizes(reduced_np)
    print(f"Without poisson encoding: {uncompressed_size*len(fifo_sizes_np)/sum(fifo_sizes_np)}")

    print(f'Before merging (with poisson): {uncompressed_size*len(reduced)/sum(reduced)}')


### Estimated using pilatus data

With poisson encoding: 5.37694634255629
Without poisson encoding: 5.1183621241202815
Before merging (with poisson): 5.680420854097133


### Estimated using ptychography data

With poisson encoding: 29.165704690943436
Without poisson encoding: 25.16935337801123
Before merging (with poisson): 30.05043073081478


### Estimated using xpcs data

With poisson encoding: 8.133659331703342
Without poisson encoding: 7.61723035825025
Before merging (with poisson): 8.514478626155174
